In [ ]:
from langchain_community.document_loaders import TextLoader #vai converter o texto em um formato que o Lang Chain possa trabalhar
from langchain_text_splitters import CharacterTextSplitter #vai dividir o texto em diferentes chunks de caracteres (há diferentes formas de fazer isso. Vale à pena pesquisar)
from langchain_openai import OpenAIEmbeddings #vai converter o texto em embeddings (vetores) que o Lang Chain possa trabalhar utilizando modelos do OpenAI
from langchain_chroma import Chroma #vaii guardar os vectors em vectors databases

In [ ]:
# Vamos agora utilizar as chaves de API do OpenAI. Parta isso vamos utilizar a a biblioteca dotenv
# com .env

from dotenv import load_dotenv

load_dotenv()


In [ ]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')
display(books)

In [ ]:
display(books["tag_description"])

#A tag é uma ótima forma de indentificar o nome do livro após utilizar a descrição para treinar o modelo, pois o que vamos ter de retorno na busca de vetores 
# é a descrição com a tag do livro. Utilizar a descrição para filtrar pelo livro é muito devagar, por isso utilizar a tag é uma melhor opçção

#vamos então extrair as descrições e as tags dos livros

books["tag_description"].to_csv("books_tag_description.csv", index=False, sep = "\n", header=False)


In [ ]:
raw_documents = TextLoader("books_tag_description.csv").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap = 0, separator = "\n") #vamos utilizar chunk_size = 0 para garantir que ele vai fazer a separação baseado no separator 
# e não em chunks de caracteres e utilizar chunk_overlap = 0 para garantir que ele não vai haver overlap entre as descrições.
documents = text_splitter.split_documents(raw_documents)


In [ ]:
#checando abaixo percebe-se que funcionou corretamente a separação das descrições
documents[0]

In [ ]:
# vamos agora criar os embeddings para cada descrição e salvá-los em um banco de dados de vetores

from langchain_huggingface import HuggingFaceEmbeddings
huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=huggingface_embeddings
)